1. define an application's workflow as a graph, where each step an agent takes is a node and the connections between steps are edges.

**2. Reflection and Refinement in LangGraph**

LangGraph allows us to build agentic systems as cyclical graphs. This is perfect for the reflection and refinement pattern, where an initial plan is generated and then passed to another "critic" agent (or the same agent with a different prompt) to critique and improve it. The process can loop until the plan meets a certain quality standard.

In [ ]:
# Install necessary libraries
!pip install -qU langchain langchain_openai python-dotenv langgraph autogenstudio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46

In [ ]:
# Import libraries and configure the API key
import os
from google.colab import userdata
from dotenv import load_dotenv

# Load environment variables (optional, for local development)
load_dotenv()

# Configure the OpenAI API key for Colab
# In Colab, go to the "Secrets" tab (key icon on the left) and add a new secret
# named OPENAI_API_KEY with your API key as the value.
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    # Fallback for local execution if you have a .env file
    if "OPENAI_API_KEY" not in os.environ:
        print("OPENAI_API_KEY not found. Please set it as a Colab secret or in your environment.")

In [ ]:
import operator
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# 2. Define the State
# The state is the central object that is passed between nodes in the graph.
class AgentState(TypedDict):
    task: str
    plan: str
    critique: str
    iteration_count: int

# 3. Define the Graph Nodes (Planner and Critic)
llm = ChatOpenAI(model="gpt-4", temperature=0)

def planner_node(state: AgentState):
    """Generates the initial or refined travel plan."""
    print("---PLANNER NODE---")
    prompt = f"""
    Based on the following task, create a detailed 5-day business trip itinerary.
    Task: {state['task']}
    Previous Plan (if any for refinement): {state['plan']}
    Critique (if any): {state['critique']}
    Generate a concise, point-by-point itinerary.
    """
    response = llm.invoke(prompt)
    state['plan'] = response.content
    state['iteration_count'] = state.get('iteration_count', 0) + 1
    return state

def critique_node(state: AgentState):
    """Critiques the plan for logical errors or improvements."""
    print("---CRITIQUE NODE---")
    prompt = f"""
    You are a critical travel assistant. Review the following travel plan for feasibility, efficiency, and completeness.
    Are the travel times logical? Is there enough time for meetings? Are buffer times included?
    Plan: {state['plan']}
    Provide constructive feedback. If the plan is good, simply say 'The plan is excellent and requires no changes.'
    """
    response = llm.invoke(prompt)
    state['critique'] = response.content
    return state

# 4. Define the Graph Edges and Conditional Logic
def should_continue(state: AgentState):
    """Decides whether to end the process or continue refining."""
    print("---CHECKING CONDITION---")
    if state['iteration_count'] > 3: # Exit after 3 iterations
        return "end"
    if "no changes" in state['critique'].lower():
        return "end"
    else:
        return "continue"

# 5. Build and Run the Graph
workflow = StateGraph(AgentState)

workflow.add_node("planner", planner_node)
workflow.add_node("critique", critique_node)

workflow.set_entry_point("planner")

workflow.add_edge("planner", "critique")
workflow.add_conditional_edges(
    "critique",
    should_continue,
    {
        "continue": "planner",
        "end": END
    }
)

app = workflow.compile()

# Execute the graph
task = """
Plan a 5-day official business trip for a manager, starting from Delhi.
Visits to Bangalore (2 days) and Hyderabad (2 days).
Purpose: Team meetings and key discussions.
Start Date: Monday, October 6th, 2025.
End Date: Friday, October 10th, 2025.
"""

initial_state = {"task": task, "plan": "", "critique": ""}
final_state = app.invoke(initial_state)

print("\n\n--- FINAL ITINERARY (Refined) ---")
print(final_state['plan'])

---PLANNER NODE---
---CRITIQUE NODE---
---CHECKING CONDITION---
---PLANNER NODE---
---CRITIQUE NODE---
---CHECKING CONDITION---
---PLANNER NODE---
---CRITIQUE NODE---
---CHECKING CONDITION---
---PLANNER NODE---
---CRITIQUE NODE---
---CHECKING CONDITION---


--- FINAL ITINERARY (Refined) ---
Day 1: Monday, October 6th, 2025 - Delhi to Bangalore
- 5:30 AM: Depart from home to Delhi Airport.
- 8:00 AM: Flight from Delhi to Bangalore.
- 11:00 AM: Arrive in Bangalore, check-in at the hotel.
- 12:00 PM: Lunch at the hotel.
- 1:00 PM - 5:00 PM: Team meeting at Bangalore office (Location to be confirmed).
- 5:00 PM - 5:30 PM: Break.
- 5:30 PM - 6:30 PM: Wrap-up and feedback session.
- 7:00 PM: Dinner with Bangalore team (Location to be confirmed).
- 9:00 PM: Free time for relaxation.

Day 2: Tuesday, October 7th, 2025 - Bangalore
- 8:00 AM: Breakfast at the hotel.
- 9:00 AM - 12:00 PM: Key discussions with Bangalore team.
- 12:00 PM - 1:00 PM: Lunch with team members.
- 1:00 PM - 4:00 PM: Cont